In [ ]:
!unzip archive.zip -d archive

In [80]:
import os
import langchain
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI,GoogleGenerativeAI

from langchain.document_loaders import DirectoryLoader, TextLoader
import matplotlib.pyplot as plt
from langchain import hub
from sentence_transformers import SentenceTransformer
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv

In [77]:
from IPython.display import display, HTML

display(HTML("<style>.container { width:100% !important; } .code_cell pre { white-space: pre-wrap; }</style>"))


In [72]:
load_dotenv()
API_KEY=os.getenv('API_KEY')

In [34]:

loader = DirectoryLoader(f'{os.getcwd()}/archive/Culture/', loader_cls=TextLoader,loader_kwargs={'encoding':'utf-8'})
# Load all documents from the directory
documents = loader.load()

In [36]:
documents[0].page_content

'ينظم معهد الشارقة للفنون معرضاً فنياً تحت عنوان باقة الفن، وذلك عند الساعة السابعة من مساء اليوم في مقر المعهد في منطقة الفنون في حي الشويهين في الشارقة، وتتلاقى في المعرض إبداعات 62 طالباً من المنتسبين للدراسة في المعهد في كافة التخصصات الفنية .'

In [55]:
class Embedding:
  def __init__(self):
    self.model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
  def embed_documents(self,text):
    return self.model.encode(text).tolist()
  def embed_query(self,query):
    return self.model.encode(query).tolist()

In [147]:
from langchain_chroma import Chroma
ahmed = Chroma(persist_directory='./vector_datastore/',embedding_function=Embedding())
ahmed_retriver = ahmed.as_retriever(search_type='similarity',search_kwargs={'k':2})
ahmed_retriver.invoke('الشارقة للفنون')

[]

In [38]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1500,chunk_overlap=150, add_start_index=True)
all_splitts = splitter.split_documents(documents)

In [126]:
embedding = Chroma(persist_directory='./vector_datastore').from_documents(embedding=Embedding(),documents=all_splitts)

d:\projects\electropi\ML\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [58]:
retriever = embedding.as_retriever(search_type='similarity',search_kwargs={'k':2})

In [116]:
retriever.invoke('الشارقة للفنون')

[Document(metadata={'source': 'd:\\projects\\electropi\\RAG\\archive\\Culture\\0218.txt', 'start_index': 0}, page_content='ينظم معهد الشارقة للفنون التابع لإدارة الفنون في دائرة الثقافة والإعلام في الشارقة معرضاً فنياً بعنوان ارتقاء ويضم مجموعة من أعمال التصوير المائي والزيتي للفنانين أحمد ضميرية وفارس طباع، وذلك عند السابعة من مساء اليوم .'),
 Document(metadata={'source': 'd:\\projects\\electropi\\RAG\\archive\\Culture\\0235.txt', 'start_index': 1354}, page_content='وإطلاق العديد من المشاريع الفنية المتميزة، ونوهت كذلك بالمناخ التعليمي ودور المغفور له بإذن الله الشيخ زايد بن سلطان آل نهيان، رحمه الله، بتشجيع الفن والفنانين، وإصراره دائماً على أن تكون كليات الفنون ضمن الجامعات التي أنشأت لخلق جيل مبدع متميز، قادر على تسجيل تراث البلد وقادر على رسم مجتمعها وأفرادها وإنجازاتها وكذلك جيل مقبل على الفن وعلومه بشتى أنواعها.وقالت د. القباني «يشكل هذا المهرجان دليلاً إضافياً على اهتمام الإمارات العربية بالفن والفنون، وحرصها الأكيد على استمرار دعم الحركة الفنية العربية، ورعايتها من خلال استضاف

In [110]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [74]:
model = GoogleGenerativeAI(google_api_key=API_KEY,model='gemini-1.5-flash')

In [106]:
prompt = hub.pull("rlm/rag-prompt",api_key=API_KEY) 
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [113]:
template = '''
    انت مساعدي في الذكاء الاصطناعي سأعطيك بعض الأسئلة و بناء على المعلومات التي لديك قم بالاجابه 
    السؤال : {question}
    السياق : {context}
    الإجابه : 
'''
prompt = PromptTemplate(template=template,input_variables=['question','context'])

In [114]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [117]:
rag_chain.invoke('ماذا ينظم معهد الشارقة للفنون')

2


'ينظم معهد الشارقة للفنون **معرضين فنيين**:\n\n* **معرض "باقة الفن"**: يضم أعمال 62 طالباً من المنتسبين للدراسة في المعهد في كافة التخصصات الفنية.\n* **معرض "ارتقاء"**: يضم أعمال التصوير المائي والزيتي للفنانين أحمد ضميرية وفارس طباع. \n'